In [1]:
import pandas as pd
import numpy as np

import neologdn
import MeCab

import re

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

C:\Users\Tomori kengo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### 日誌サンプルデータの読み込み

In [2]:
#encoding='utf-8'で上手くいった
diary_df = pd.read_csv('SBI_Financial statement_201903.csv', encoding='utf-8')

In [3]:
diary_df

,text
0,国内経済が緩やかに回復するなか、マーケット環境は、米国と中国の貿易摩擦問題に対する警戒感等か...
1,このような環境の中、当社業績においては、ホールセールビジネスの拡大、トレーディング収益や金融...
2,当社は引続き他社を大きく上回る高いシェアを維持し、35.9％のシェアを獲得。
3,先物・オプションの委託個人売買代金シェアは、引き続き高水準を維持。
4,投資信託残高の四半期末残高は過去最高を更新し、信託報酬は高水準を維持。
5,2018年4月から2018年6月までの上場会社数は20社。
6,同期間のSBI証券引受関与率は100％と 引き続き業界トップ。


### MeCabにかける前準備としてneologdn.normalize()を使用して文章全体を正規化

In [4]:
def get_diary_normalization(text):
    diary_normalization = neologdn.normalize(text)
    return diary_normalization

### MeCab + neologdで形態素解析し、名詞、形容詞原形と動詞原形を抽出しリストに格納
##### 参考：https://github.com/kujirahand/book-mlearn-gyomu/blob/master/src/ch4/Doc2Vec/create_model.py

In [5]:
neologd_tagger = MeCab.Tagger('-Ochasen -d C:\mecab-ipadic-neologd')

# 引数のテキストを分かち書きして配列にする
# node.surface: 文字のみ取得できる 出力例：同  期間  の  SBI  証券
# node.feature: 品詞、原形などの詳細を取得できる:「品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音」の順
# よって、「品詞」を取得したい場合はnode.featureをsplit()でリスト型にした後に抽出したい詳細のインデックス番号[0]を指定すればよい
# また、動詞や形容詞の「原形（の単語）」を取得したい場合はインデックス番号[6]を指定すればよい

def split_words(diary_normalization):
    node = neologd_tagger.parseToNode(diary_normalization) #parseだとエラー「 'str' object has no attribute 'feature'」
    wakati_words = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if  hinshi in ["名詞"]:
            wakati_words.append(node.surface)
        elif hinshi in ["動詞", "形容詞"]:
            wakati_words.append(node.feature.split(",")[6])
        node = node.next
    return wakati_words

In [6]:
wakati_list = []

# 文章の上から順に作成した関数を実行
for diary in diary_df['text']:
    diary_normalization = get_diary_normalization(diary)
    wakati_words = split_words(diary_normalization)
    wakati_list.append(wakati_words)
    
wakati_list

[['国内',
  '経済',
  '緩やか',
  '回復',
  'する',
  'なか',
  'マーケット',
  '環境',
  '米国',
  '中国',
  '貿易摩擦',
  '問題',
  '警戒感',
  '等',
  '一進一退',
  '展開'],
 ['よう',
  '環境',
  '中',
  '当社',
  '業績',
  'ホールセール',
  'ビジネス',
  '拡大',
  'トレーディング',
  '収益',
  '金融',
  '収益',
  '増加',
  '前年同期',
  '比',
  '増収増益',
  '達成'],
 ['当社', '他社', '大きい', '上回る', '高い', 'シェア', '維持', 'する', '35.9%', 'シェア', '獲得'],
 ['先物', 'オプション', '委託', '個人', '売買代金', 'シェア', '高水準', '維持'],
 ['投資信託', '残高', '四半期', '末', '残高', '過去最高', '更新', 'する', '信託報酬', '高水準', '維持'],
 ['2018年', '4月', '2018年', '6月', '上場会社', '数', '20', '社'],
 ['期間', 'SBI証券', '引受', '関与', '率', '100%', '業界', 'トップ']]

In [7]:
trainings_wakati_list = [TaggedDocument(words = wakati_list[i], tags = [i]) for i in range(len(diary_df))]
trainings_wakati_list

[TaggedDocument(words=['国内', '経済', '緩やか', '回復', 'する', 'なか', 'マーケット', '環境', '米国', '中国', '貿易摩擦', '問題', '警戒感', '等', '一進一退', '展開'], tags=[0]),
 TaggedDocument(words=['よう', '環境', '中', '当社', '業績', 'ホールセール', 'ビジネス', '拡大', 'トレーディング', '収益', '金融', '収益', '増加', '前年同期', '比', '増収増益', '達成'], tags=[1]),
 TaggedDocument(words=['当社', '他社', '大きい', '上回る', '高い', 'シェア', '維持', 'する', '35.9%', 'シェア', '獲得'], tags=[2]),
 TaggedDocument(words=['先物', 'オプション', '委託', '個人', '売買代金', 'シェア', '高水準', '維持'], tags=[3]),
 TaggedDocument(words=['投資信託', '残高', '四半期', '末', '残高', '過去最高', '更新', 'する', '信託報酬', '高水準', '維持'], tags=[4]),
 TaggedDocument(words=['2018年', '4月', '2018年', '6月', '上場会社', '数', '20', '社'], tags=[5]),
 TaggedDocument(words=['期間', 'SBI証券', '引受', '関与', '率', '100%', '業界', 'トップ'], tags=[6])]

In [9]:
model = Doc2Vec(documents=trainings_wakati_list, size=100, min_count=1, dm=0)

C:\Users\Tomori kengo\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [10]:
print(model.docvecs[0])

[ 2.58307229e-03 -2.87590036e-03  1.84542802e-03  3.79760226e-04
 -3.63458367e-03  1.62870798e-03  2.90631113e-04 -3.76525009e-03
  4.19726013e-04  7.41325435e-04 -1.40767696e-03  3.44016519e-03
 -3.48104659e-04  4.07948624e-03 -4.62286454e-03  4.67364630e-03
 -4.97925142e-03 -1.16653298e-03  1.45586743e-03 -4.39416012e-03
 -9.75691772e-04  2.77430099e-03  3.29152821e-03  1.62896793e-03
 -1.96863874e-03 -4.33021598e-03 -3.45873134e-03  3.67706106e-03
 -1.87462289e-03 -7.34648711e-05 -1.94591575e-03 -4.40182025e-03
  2.14161072e-03 -4.20969538e-03 -1.44155475e-03 -2.36744923e-03
  3.26654967e-03  3.81742767e-03 -4.32078587e-03  4.14358499e-03
  4.49732586e-04  6.37035526e-04 -1.71470642e-03 -2.86701485e-03
  4.10954189e-03  4.63616010e-03 -1.54243107e-03  4.60500922e-03
 -2.54580693e-04  3.16397217e-03  1.48711249e-03  4.55835555e-03
 -4.45452053e-03  1.73137465e-03  4.81632585e-03 -4.57772799e-03
 -2.17327126e-03 -1.75905030e-03 -2.76785716e-03 -1.91723299e-03
 -4.81358264e-03 -1.02819

In [17]:
for i in range(len(diary_df)):
    print('tag = ' + str(i) +'の文章との類似度{}'.format(model.docvecs.most_similar(i)))

tag = 0の文章との類似度[(5, 0.2770974636077881), (1, 0.07727166265249252), (6, 0.07696818560361862), (3, 0.028534501791000366), (2, 0.01765500009059906), (4, -0.07608857750892639)]
tag = 1の文章との類似度[(6, 0.16894283890724182), (0, 0.07727167010307312), (5, -0.003815270960330963), (3, -0.05440355837345123), (2, -0.0580289289355278), (4, -0.1821083128452301)]
tag = 2の文章との類似度[(4, 0.056835103780031204), (0, 0.017655007541179657), (6, -0.04060372710227966), (3, -0.05082041025161743), (1, -0.058028921484947205), (5, -0.09862123429775238)]
tag = 3の文章との類似度[(0, 0.028534486889839172), (5, -0.022549912333488464), (4, -0.04953721910715103), (2, -0.05082041025161743), (1, -0.05440356209874153), (6, -0.08479255437850952)]
tag = 4の文章との類似度[(2, 0.05683509260416031), (6, -0.023323191329836845), (3, -0.049537211656570435), (5, -0.0689239650964737), (0, -0.07608857750892639), (1, -0.18210828304290771)]
tag = 5の文章との類似度[(0, 0.2770974338054657), (6, 0.12971948087215424), (1, -0.003815263509750366), (3, -0.02254991233348